# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-1. Применяем простейший локальный поиск.

В этой задаче Вам предлагается попробовать стандартную технику локального поиска (local search) в применении к задаче о сбалансированном разбиении графа. Мы будем рассматривать безвесовый вариант задачи с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* [Скачайте](http://mat.gsia.cmu.edu/COLOR/instances.html#XXMYC) файлы mycielX.col  (`for X in range(1,8)`).  (Если интересно, откуда такие графы берутся и чем интересны, см. конструкцию Зыкова—Мыцельского [здесь](https://docs.com/dainiak/3327).)
* Для каждого из графов найдите локальным поиском локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Ваша функция `basic_local_search` должна принимать на вход граф в формате, предоставляемом функцией `read_col_file`, и возвращать найденное локально минимальное разбиение просто как множество либо список вершин, лежащих в одной любой из двух компонент разбиения.

In [199]:
import random
from itertools import permutations
from copy import copy
import numpy as np
from tqdm import tqdm

In [74]:
def read_col_file(filename):
    with open(filename, 'r') as file:
        vertices, edges = set(), set()
        for line in file:
            line = line.strip()
            if line.startswith('p'):
                vertices = set(range(1, int(line.split()[-2]) + 1))
            elif line.startswith('e'):
                edges.add(tuple(map(int, line.split()[-2:])))
        return (vertices, edges)

In [202]:
def getCutCapacity(graph, cut):
    ans = 0
    for edge in graph[1]:
        if edge[0] in cut[0] and edge[1] in cut[1]:
            ans += 1
            continue
        if edge[1] in cut[0] and edge[0] in cut[1]:
            ans += 1
            continue
    return ans

def getRandomCut(graph, alpha):
    cut = (set(graph[0]), set())
    while len(cut[0]) / len(graph[0]) > alpha:
        vertex = random.sample(cut[0], 1)[0]
        cut[0].remove(vertex)
        cut[1].add(vertex)
    return cut

def getBestInKLocal(graph, cut, k):
    test = copy(cut)
    best = copy(cut)
    best_score = getCutCapacity(graph, cut)
    for first in permutations(cut[0], k):
        for second in permutations(cut[1], k):
            fs = set(first) | set(second)
            for i in permutations(fs, k):
                fadd = set(i)
                frem = copy(fs).difference(fadd)
                test[0].union(fadd).difference(frem)
                test[1].union(frem).difference(fadd)
                now_score = getCutCapacity(graph, test)
                if now_score <= best_score:
                    best_score = now_score
                    best = copy(test)
    return best

In [203]:
def basic_local_search(graph, k):
    cut = getRandomCut(graph, 0.5)
    now = getCutCapacity(graph, cut)
    while True:
        cut = getBestInKLocal(graph, cut, k)
        new_score = getCutCapacity(graph, cut)
        if new_score >= now:
            break
        now = new_score
    return cut

К сожалению удалось найти не все файлы графов, на указанном сайте есть только с 3 по 7.

In [212]:
def operateGraphs(k):
    for i in range(3, 8):
        name = 'myciel' + str(i) + '.col'
        g = read_col_file(name)
        cut = basic_local_search(g, k)
        print('answer for',name, 'with', k, 'local search :')
        print(getCutCapacity(g, cut), cut)
        print()

In [213]:
operateGraphs(1)

answer for myciel3.col with 1 local search :
11 ({1, 4, 7, 8, 9}, {2, 3, 5, 6, 10, 11})

answer for myciel4.col with 1 local search :
33 ({3, 7, 8, 10, 11, 13, 14, 15, 20, 21, 22}, {1, 2, 4, 5, 6, 9, 12, 16, 17, 18, 19, 23})

answer for myciel5.col with 1 local search :
122 ({3, 4, 5, 6, 12, 14, 16, 19, 20, 21, 24, 26, 27, 29, 30, 31, 32, 33, 35, 36, 37, 39, 46}, {1, 2, 7, 8, 9, 10, 11, 13, 15, 17, 18, 22, 23, 25, 28, 34, 38, 40, 41, 42, 43, 44, 45, 47})

answer for myciel6.col with 1 local search :
395 ({2, 3, 4, 6, 7, 8, 9, 15, 19, 22, 23, 25, 26, 27, 30, 31, 33, 34, 35, 37, 38, 40, 43, 44, 48, 52, 54, 58, 60, 62, 63, 65, 67, 68, 69, 70, 71, 72, 73, 74, 76, 79, 81, 82, 87, 90, 92}, {1, 5, 10, 11, 12, 13, 14, 16, 17, 18, 20, 21, 24, 28, 29, 32, 36, 39, 41, 42, 45, 46, 47, 49, 50, 51, 53, 55, 56, 57, 59, 61, 64, 66, 75, 77, 78, 80, 83, 84, 85, 86, 88, 89, 91, 93, 94, 95})

answer for myciel7.col with 1 local search :
1162 ({1, 3, 4, 6, 7, 8, 9, 12, 13, 14, 17, 18, 19, 21, 24, 26, 27, 3

Локальный 2 поиск был запущен на час, но не успел завершиться

In [214]:
operateGraphs(2)

answer for myciel3.col with 2 local search :
10 ({2, 4, 5, 6, 9}, {1, 3, 7, 8, 10, 11})

answer for myciel4.col with 2 local search :
44 ({1, 3, 6, 7, 8, 9, 10, 11, 13, 19, 23}, {2, 4, 5, 12, 14, 15, 16, 17, 18, 20, 21, 22})

answer for myciel5.col with 2 local search :
113 ({1, 2, 3, 4, 9, 10, 16, 17, 18, 22, 23, 25, 27, 32, 33, 35, 38, 39, 41, 42, 44, 45, 47}, {5, 6, 7, 8, 11, 12, 13, 14, 15, 19, 20, 21, 24, 26, 28, 29, 30, 31, 34, 36, 37, 40, 43, 46})



KeyboardInterrupt: 

Немного статистики на примере 'myciel3.col'. Сравнение 1, 2 и 3 локального поиска.

In [218]:
g = read_col_file('myciel3.col')
stat = np.zeros((3, 30))
for i, k in enumerate([1, 2, 3]):
    for j in tqdm(range(stat.shape[1])):
        cut = basic_local_search(g, k)
        stat[i][j] = (getCutCapacity(g, cut))
        
print('Полученные оценки')
print(stat)
print('Среднее')
print(stat.mean(axis=1))

100%|██████████| 30/30 [07:11<00:00, 14.25s/it]

Полученные оценки
[[ 10.  11.  11.  10.  11.   9.  12.   8.  15.  12.  10.  14.  12.   9.
    9.  16.   9.  13.   8.  11.  11.  13.  13.  15.  14.  16.   9.  11.
   16.   8.]
 [ 13.  10.   9.  12.  12.   9.   9.  10.  12.  11.  11.  15.  10.   9.
   13.  11.  13.  12.  10.  10.  13.  11.  14.   9.   9.   9.   8.   8.
    8.  15.]
 [  8.  14.  11.  11.  16.  11.  10.  11.  10.  10.  12.  13.   9.   8.
   10.  10.   9.  10.  10.  15.  13.   9.  12.  14.   9.  12.   8.   9.
   16.  11.]]
Среднее
[ 11.53333333  10.83333333  11.03333333]


Учитывая, что локальный 3 поиск работает раз в 400 дольше чем 2 поиск, а его результаты даже хуже, кажется, что в этой задаче он бесполезен.